In [1]:
%matplotlib inline
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, execute, Aer, IBMQ, QuantumRegister, ClassicalRegister
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *

# Loading your IBM Q account(s)
provider = IBMQ.load_account()

In [4]:
import numpy as np
import math
from sklearn.model_selection import train_test_split
from sklearn import datasets
from qiskit.circuit.library import MCXGate
import matplotlib.pyplot as plt
import random

In [3]:
def cnx(qc,*qubits):
    """Control n-1 qubits, apply 'not' to last one
    Follows:
    @article{PhysRevA.52.3457,
      title = {Elementary gates for quantum computation},
      author = {Barenco, Adriano and Bennett, Charles H. and Cleve, Richard and DiVincenzo, David P. and Margolus, Norman and Shor, Peter and Sleator, Tycho and Smolin, John A. and Weinfurter, Harald},
      doi = {10.1103/PhysRevA.52.3457},
      url = {https://link.aps.org/doi/10.1103/PhysRevA.52.3457}
    }
    Follwing Lemma 7.9, which uses Lemma 5.1 and 4.3
    """
    if len(qubits) >= 3:
        last = qubits[-1]
        #A matrix: (made up of a  and Y rotation, lemma4.3)
        qc.crz(np.pi/2,qubits[-2],qubits[-1])
        #cry
        qc.cry(qc,np.pi/2,qubits[-2],qubits[-1])
        
        #Control not gate
        qc.cnx(qc,*qubits[:-2],qubits[-1])
        
        #B matrix (cry again, but opposite angle)
        qc.cry(qc,-np.pi/2,qubits[-2],qubits[-1])
        
        #Control
        qc.cnx(qc,*qubits[:-2],qubits[-1])
        
        #C matrix (final rotation)
        qc.crz(-np.pi/2,qubits[-2],qubits[-1])
    elif len(qubits)==3:
        qc.ccx(*qubits)
    elif len(qubits)==2:
        qc.cx(*qubits)

In [4]:
def encode(x):
    n = len(x)
    binary = ''
    for i in range(2):
        temp = int(x[i]/2)
        tempBin = bin(temp).replace("0b", "")
        if len(tempBin) < 2:
            tempBin = '0'*(2-len(tempBin)) + tempBin
        binary += tempBin
    return binary

In [6]:
test_data_points = 1
data_size = 8
exponent = int(math.log(data_size, 2))
data = np.array(np.arange(data_size), dtype= np.uint8)
label = np.zeros(data_size)
label[1::2] = 1 
data= np.flip((((data[:,None] & (1 << np.arange(exponent)))) > 0).astype(int), axis=1)

trainData,testData,trainLabel,testLabel = train_test_split(data,label,test_size=test_data_points)    
print("Training data points:")
print(trainData)
print("Training Label: ")
print(trainLabel)
print("Test data point:")
print(testData)

[0 1 2 3 4 5 6 7]
Training data points:
[[0 0 1]
 [0 1 0]
 [1 0 1]
 [1 1 1]
 [0 1 1]
 [1 1 0]
 [0 0 0]]
Training Label: 
[1. 0. 1. 1. 1. 0. 0.]
Test data point:
[[1 0 0]]


In [8]:
p = len(trainLabel)
n = len(testData[0])
v = QuantumRegister(p,"v")
c = QuantumRegister(p,"c")
measure = ClassicalRegister(p,"measure")
T = QuantumCircuit(v,c,measure)
T.h(v)
for i in range(p):
    #for j in range(n):
    #    if trainData[i][j] == 1:
    #        T.x(v[n*i + j])
    if trainLabel[i] == 1:
        T.x(c[i])
T.barrier()
T.h(v)
T.h(c)
c_qubits = p
v_qubits = p

In [9]:
test = QuantumRegister(n,"x")
x = QuantumCircuit(test)
for i in range(n):
    if testData[0][i] == 1:
        x.x(test[i])
n_qubits = p + p + n
x_qubits = n
x.barrier()

total qubits:  17


In [10]:
main_circ = x + T
anc = QuantumRegister(1,"anc")
main_circ.add_register(anc)
n_qubits += 1
anc_qubits = 1
main_circ.draw()

total qubits:  18


In [11]:
[x,v,c,anc] = main_circ.qregs
main_circ.x(x[0])

In [12]:
for i in range(p):
    for j in range(n):
        main_circ.cx(x[j],v[i])

In [13]:
main_circ.barrier()
a = QuantumRegister(n,"a")
o = QuantumRegister(1,"o")
main_circ.add_register(a)
main_circ.add_register(o)

a_qubits = n
o_qubits = 1
n_qubits += n + 1

total qubits:  22


In [14]:
for i in range(p):
    for j in range(n):
        if (j > 0):
            main_circ.ccx(v[i],o[0],a[j])
            main_circ.x(a[:j])
            mcx = MCXGate(num_ctrl_qubits=j+2)
            #main_circ.mcx([v[i],a[:j]],o)
            cnx(main_circ,[v[i],a[:j],o])
            main_circ.x(a[:j])
        else:
            main_circ.ccx(v[i],o[0],a[j])
            main_circ.ccx(v[i],a[j],o[0])
main_circ.x(o)
main_circ.barrier()

In [15]:
threshold = 5
t = math.ceil(np.log2(threshold))
orQubits = QuantumRegister(t,"orRegister")
main_circ.add_register(orQubits)
main_circ.x(a)
main_circ.x(o)
for i in range(0,2,t):
    main_circ.ccx(a[n-2-i],a[n-1-i],orQubits[i])
main_circ.ccx(o,orQubits[0],orQubits[t-1])
main_circ.x(orQubits[n-1])

or_qubits = t
n_qubits += n
print("total qubits: ",n_qubits)

total qubits:  25


In [16]:
main_circ.measure(c,measure)

In [17]:
from statistics import mode
import operator

shots = 11
backend = Aer.get_backend('qasm_simulator')
results = execute(main_circ, backend=backend, shots=shots).result()
count = results.get_counts()
mx = list(count.keys())
classes = {'0':0,'1':0}

for i in range(len(mx)):
    label = mx[i]
    #print(label)
    lab_dict = {'0':0,'1':0}
    for element in label:
        lab_dict[element] += 1
    maxFreq = -1
    y_pred = -1
    print("lab_dict: ",lab_dict)
    for key in lab_dict:
        if lab_dict[key] > maxFreq:
            maxFreq = lab_dict[key]
            y_pred = key
    classes[y_pred] += 1
print(classes)
freq = 0
total_freq = 0
for keys in classes:
    total_freq += classes[keys]
    if classes[keys] > freq:
        freq = classes[keys]
        Y_pred = keys

print("Predicted class label:",Y_pred)

lab_dict:  {'0': 5, '1': 2}
lab_dict:  {'0': 4, '1': 3}
lab_dict:  {'0': 5, '1': 2}
lab_dict:  {'0': 4, '1': 3}
lab_dict:  {'0': 3, '1': 4}
lab_dict:  {'0': 5, '1': 2}
lab_dict:  {'0': 4, '1': 3}
lab_dict:  {'0': 1, '1': 6}
lab_dict:  {'0': 4, '1': 3}
lab_dict:  {'0': 2, '1': 5}
lab_dict:  {'0': 4, '1': 3}
{'0': 8, '1': 3}
Predicted class label: 0
